# Analysis Walkthrough

## Determine all of the parameters

### Specify the locations of the files for X_train, X_test, and y_train. Also, there is a file that contains information about the individual stations that can be useful for models that learn for each station.

In [1]:
import os
os.chdir('..')

In [32]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'

### Import the various files. This is mostly done so that any file updates during testing are carried over to this notebook.

In [33]:
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model
import solar.report.submission
import numpy as np

### Set the parameters that will be used to set up the data. There are some parameters that determine the size and shape of the data but have effects other than setting up feature columns. This includes the dates that are included for testing and training, the stations considered, and whether to have X values correspond to a date or to a specific date/station combination.

In [34]:
# Choose up to 98 stations; not specifying a station means to use all that fall within the given lats and longs. If the
# parameter 'all' is given, then it will use all stations no matter the provided lats and longs
#station = ['ACME', 'BEAV', 'ADAX']
station = ['all']

# Determine which dates will be used to train the model. No specified date means use the entire set from 1994-01-01 
# until 2007-12-31.
#train_dates = ['1994-01-01','2007-12-31']
train_dates = ['1994-01-01','1994-03-01']
# Determine the test X values to produce. There is no practical purpose to use fewer than all of the points other than
# for testing. Again, not choosing a date will use 2008-01-01 through 2012-11-30.
test_dates = ['2008-01-01', '2008-01-05']
#test_dates = ['all']
#test_dates = []
#train_dates = []
#test_dates = []

# The last parameter that is not specifically involved in feature selection in the layout to be used for training
# I have switched to almost always training for each individual station rather than having a single row for a date.
# However, I am still not beating the benchmark, and the file would grow too large to have the benchmark laid out
# with a row for each station, so I'll keep the switch. True means that each station has a row (5113 dates X 98
# stations to train the data). False means that there are 5113 rows that are being used to train the data.
station_layout = True

### First, just duplicate the functionality of the basic grid analysis

In [35]:
# Use all variables
#var = ['uswrf_sfc', 'dswrf_sfc']
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0]

# Aggregate over all times
times = [12, 15, 18, 21, 24]

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
just_grid = [default_grid]


### Run data extraction

In [6]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

%prun input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, just_grid)

INFO:solar.wrangle.wrangle:Started building test and training data
INFO:solar.wrangle.wrangle:Features: [{'axes': {'var': ['uswrf_sfc', 'dswrf_sfc'], 'models': [0, 7], 'station': ['ACME', 'BEAV', 'ADAX'], 'times': [18, 21]}, 'type': 'relative'}]
INFO:solar.wrangle.wrangle:Train dates: ['2005-11-30', '2006-11-30']
INFO:solar.wrangle.wrangle:Test dates: ['2008-12-29', '2009-02-04']
INFO:solar.wrangle.wrangle:Stations: ['ACME', 'BEAV', 'ADAX']
INFO:solar.wrangle.wrangle:Finished building test and training data


In [8]:
#input_data[3]

### Run through the full analysis

In [10]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'
cv_splits = 10

model = Model.model(input_data, Ridge, {'alpha':np.logspace(-3,1,8,base=10)}, cv_splits, 
                    error_formula, normalize=True)

INFO:solar.analyze.model:Started building model
INFO:solar.analyze.model:Train input columns: MultiIndex(levels=[[u'relative'], [0, 7], [u'dswrf_sfc', u'uswrf_sfc'], [u'NE', u'NW', u'SE', u'SW'], [18, 21]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
           names=[None, u'models', u'var', u'lat_longs', u'times'])
INFO:solar.analyze.model:Train input indices: [[2005-11-30 00:00:00, 2005-12-01 00:00:00, 2005-12-02 00:00:00, 2005-12-03 00:00:00, 2005-12-04 00:00:00, 2005-12-05 00:00:00, 2005-12-06 00:00:00, 2005-12-07 00:00:00,

In [ ]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, input_data[1], input_data[2], {'grid'})

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# here we set some aesthetic parameters so that all of our figures are nice and big
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
sns.set(style="white", context="talk")

In [ ]:
y_pred = model.predict(input_data[0])

In [ ]:
errors = abs(y_pred - input_data[1])/input_data[1]

In [ ]:
#sns.distplot(errors, rug=True)

In [ ]:
#input_data[0].shape

In [ ]:
#errors.shape

In [ ]:
#input_data[0][(errors > 0.8).values]

In [ ]:
#input_data[1][(errors > 0.8).values]

In [ ]:
#pd.DataFrame(y_pred, index=input_data[1].index, columns=input_data[1].columns)[(errors > 0.8).values]

In [ ]:
#max(errors.values)

In [ ]:
#unstacked = input_data[0].stack('time').stack('model').stack('variable').stack('lat_longs').reset_index('model').reset_index('time').reset_index('variable').reset_index('lat_longs')

In [ ]:
#dswrf = unstacked[(unstacked['lat_longs'] == 'NE') & (unstacked['variable'] == 'dswrf_sfc') & (unstacked['time'] == 21)
#         & (unstacked['model'] == 0)]['relative']

In [ ]:
#pd.DataFrame(dswrf).reset_index('station').rename(columns={'station':'location'}).unstack('location').set_index('location')

In [ ]:
#dswrf

In [ ]:
#pd.concat((errors,dswrf))

In [ ]:
#pd.concat([dswrf,errors])

In [ ]:
#sns.jointplot("total_solar","dswrf", data=drinks[(drinks.beer < 100) & (drinks.wine < 30)] , kind = "kde")

In [ ]:
#import netCDF4 as nc
#X = nc.Dataset('solar/data/kaggle_solar/train/dswrf_sfc_latlon_subset_19940101_20071231.nc','r+').variables.values()
#X[-1][0:10,0,2:3,:,:]

In [6]:
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}
stat_feats = [stat_names]

### Next, we start to layout the features to include. The two most important (and complicated) are 'absolute', which just reports out the weather variables at specific GEFS, times, and models, and 'relative' which uses a grid to identify nearby GEFS for weather measurements based on the location of the station. The second option makes the most sense when using the station_layout above, but it will work with either layout.

In [ ]:
# A very simple model would just take the average value of all variables at all locations, using all models over the
# course of the day. Here, only the var parameter and one value of the model is expanded. 
# All of the other axes are aggregated using an aggregation function. In this case, the mean value. 
# This will provide a 15 aggregated columns for model 0 and 15 aggregated columns for the mean of models 1 though 10.
# In this case, setting station_layout to false would make the most sense because the measurements will be repeated
# for each station. However, for consistency in this walkthough, I will just keep it in the station_layout.

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Aggregate over all latitudes which surround Mesonet stations (exclude those that are outside of the main grid)
lats = range(33,38)

# Same as for lats
longs = range(257,267)

all_avgs = {'type':'absolute', 'full_axes':{'var':var, 'models':model1}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]], 'lats':[lats,[np.median]],
                        'longs':[longs,[np.median]]}}

avgs_feats = [all_avgs]

In [ ]:
# A similar example using a surrounding grid for each station. There are no lat or long options for this type of
# feature set

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Create a column for each member of the grid. All or nothing for gefs now. Could specify but currently see no need
# for it. We could also take an aggregate measure of the gefs (including interpolate). That doesn't work for the
# other dimensions

gefs = ['all']

grid_avgs = {'type':'relative', 'full_axes':{'var':var, 'models':model1, 'gefs':gefs}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]]}}

grid_feats = [grid_avgs]

In [31]:
# A similar example using a surrounding grid for each station. Now, just average over the grid

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Create a column for each member of the grid. All or nothing for gefs now. Could specify but currently see no need
# for it. We could also take an aggregate measure of the gefs (including interpolate). That doesn't work for the
# other dimensions

gefs = ['all']

grid_avgs = {'type':'relative', 'full_axes':{'var':var, 'models':model1}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]], 'gefs':[gefs, [np.mean]]}}

gefs_mean_feats = [grid_avgs]

In [36]:
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}
stat_feats = [stat_names]

In [37]:
frac_dist = {'type':'frac_dist'}
dist_feats = [frac_dist]

In [52]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

dist_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, dist_feats)

INFO:solar.wrangle.wrangle:Started building test and training data
INFO:solar.wrangle.wrangle:Features: [{'type': 'frac_dist'}]
INFO:solar.wrangle.wrangle:Finished building test and training data


log/log_2016-02-06-16-04-32.log


In [38]:
days_solstice = {'type':'days_from_solstice'}
days_cold = {'type':'days_from_coldest'}
days_feats = [days_solstice, days_cold]

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

dist_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, days_feats)

In [39]:
all_feats = [frac_dist, days_cold, days_solstice, default_grid, stat_names]

In [62]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

all_feats_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, all_feats)

INFO:solar.wrangle.wrangle:Started building test and training data
INFO:solar.wrangle.wrangle:Features: [{'type': 'frac_dist'}, {'type': 'days_from_coldest'}, {'type': 'days_from_solstice'}, {'axes': {'var': ['all'], 'models': [0], 'station': ['all'], 'times': [12, 15, 18, 21, 24]}, 'type': 'relative'}, {'type': 'station_names'}]
INFO:solar.wrangle.wrangle:Train dates: ['1994-01-01', '1994-03-01']
INFO:solar.wrangle.wrangle:Test dates: ['2008-01-01', '2008-01-05']
INFO:solar.wrangle.wrangle:Stations: ['all']
INFO:solar.wrangle.wrangle:Finished building test and training data


In [63]:
all_feats_data[0]

(frac_dist, lat_dist)  (frac_dist, long_dist)  \
train_dates station                                                  
1994-01-01  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            ALTU                   0.58722                 0.66192   
            APAC                   0.91418                 0.70784   
            ARNE                   0.07204                 0.09692   
            BEAV                   0.80253                 0.46988   
            BESS                   0.40185                 0.94153   
            BIXB                   0.96305                 0.13379   
            BLAC                   0.75443                 0.74548   
            BOIS                   0.69256                 0.50287   
            BOWL                   0.17156                 0.36879   
            BREC                   0.41201                 0.30606   
            BRIS                   0.78050                 0.64596   
            BUFF                   0.83129                 0.35899   
            BURB                   0.63459                 0.18954   
            BURN                   0.89376                 0.73082   
            BUTL                   0.59150                 0.72941   
            BYAR                   0.84970                 0.99670   
            CAMA                   0.02866                 0.65348   
            CENT                   0.60896                 0.66691   
            CHAN                   0.65282                 0.19593   
            CHER                   0.74813                 0.63726   
            CHEY                   0.54615                 0.27210   
            CHIC                   0.03236                 0.08554   
            CLAY                   0.65657                 0.67404   
            CLOU                   0.22321                 0.75130   
            COOK                   0.68001                 0.15104   
            COPA                   0.90987                 0.11447   
            DURA                   0.92075                 0.67973   
            ELRE                   0.54848                 0.96346   
...                                    ...                     ...   
1994-03-01  PERK                   0.99865                 0.95169   
            PRYO                   0.36914                 0.72862   
            PUTN                   0.89904                 0.03962   
            REDR                   0.35590                 0.84694   
            RETR                   0.12275                 0.63999   
            RING                   0.19365                 0.41188   
            SALL                   0.43815                 0.20195   
            SEIL                   0.19033                 0.95970   
            SHAW                   0.36492                 0.05178   
            SKIA                   0.41530                 0.96294   
            SLAP                   0.59749                 0.73808   
            SPEN                   0.54208                 0.65854   
            STIG                   0.26527                 0.81884   
            STIL                   0.12093                 0.90473   
            STUA                   0.87642                 0.93018   
            SULP                   0.56610                 0.04952   
            TAHL                   0.97235                 0.01329   
            TALI                   0.71070                 0.98848   
            TIPT                   0.43972                 0.86245   
            TISH                   0.33262                 0.32105   
            VINI                   0.77536                 0.77906   
            WASH                   0.98224                 0.47891   
            WATO                   0.84185                 0.47385   
            WAUR                   0.16775                 0.01185   
            WEAT               

In [64]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'
cv_splits = 10

write = 'local'
njobs = 1

model = Model.model(all_feats_data, Ridge, {'alpha':np.logspace(-5,-2,8,base=10)}, cv_splits, 
                    error_formula, njobs, write, normalize=True, random_state=1)

INFO:solar.analyze.model:Started building model
INFO:solar.analyze.model:Train input columns: Index([              (u'frac_dist', u'lat_dist'),
                    (u'frac_dist', u'long_dist'),
         (u'days_from_coldest', u'from_coldest'),
       (u'days_from_solstice', u'from_solstice'),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
       ...
                                    u'stat_TISH',
                                    u'stat_VINI',
                                    u'stat_WASH',
                                    u'stat_WATO',
                                    u'stat_WAUR',
                                    u'stat_WEAT',
                                    u'stat_WEST',
                                    u'stat_WI

In [67]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
cv_splits = 10

error_formula = 'mean_absolute_error'
njobs = 1
write = 'local'
%prun model = Model.model(all_feats_data, GradientBoostingRegressor, {'n_estimators':[300]}, \
                    cv_splits, error_formula, njobs, write, loss='ls', max_depth=1, random_state=0, learning_rate=.001)

INFO:solar.analyze.model:Started building model
INFO:solar.analyze.model:Train input columns: Index([              (u'frac_dist', u'lat_dist'),
                    (u'frac_dist', u'long_dist'),
         (u'days_from_coldest', u'from_coldest'),
       (u'days_from_solstice', u'from_solstice'),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
                                (u'relative', 0),
       ...
                                    u'stat_TISH',
                                    u'stat_VINI',
                                    u'stat_WASH',
                                    u'stat_WATO',
                                    u'stat_WAUR',
                                    u'stat_WEAT',
                                    u'stat_WEST',
                                    u'stat_WI

In [61]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'
cv_splits = 10

model = Model.model_from_pickle('input_2016-02-06-15-38-35.p', Ridge, {'alpha':np.logspace(5,10,10,base=10)}, cv_splits, 
                    error_formula, normalize=True, random_state=1)

IOError: [Errno 2] No such file or directory: 'solar/data/kaggle_solar/inputs/input_2016-02-06-15-38-35.p'

In [60]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, all_feats_data[1], all_feats_data[2], True)

INFO:solar.report.submission:Started building submission file
INFO:solar.report.submission:Finished building submission file


                   frac_dist           days_from_coldest days_from_solstice  \
dist_ind            lat_dist long_dist      from_coldest      from_solstice   
test_dates station                                                            
2008-01-01 ACME      0.80833   0.97675                 3                 11   
           ADAX      0.79851   0.33091                 3                 11   
           ALTU      0.58722   0.66192                 3                 11   
           APAC      0.91418   0.70784                 3                 11   
           ARNE      0.07204   0.09692                 3                 11   
           BEAV      0.80253   0.46988                 3                 11   
           BESS      0.40185   0.94153                 3                 11   
           BIXB      0.96305   0.13379                 3                 11   
           BLAC      0.75443   0.74548                 3                 11   
           BOIS      0.69256   0.50287              

In [63]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, all_feats_data[1], all_feats_data[2], True)

INFO:solar.report.submission:Started building submission file
INFO:solar.report.submission:Finished building submission file


                   frac_dist           days_from_coldest days_from_solstice  \
dist_ind            lat_dist long_dist      from_coldest      from_solstice   
test_dates station                                                            
2008-01-01 ACME      0.80833   0.97675                 3                 11   
           ADAX      0.79851   0.33091                 3                 11   
           ALTU      0.58722   0.66192                 3                 11   
           APAC      0.91418   0.70784                 3                 11   
           ARNE      0.07204   0.09692                 3                 11   
           BEAV      0.80253   0.46988                 3                 11   
           BESS      0.40185   0.94153                 3                 11   
           BIXB      0.96305   0.13379                 3                 11   
           BLAC      0.75443   0.74548                 3                 11   
           BOIS      0.69256   0.50287              

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

stat_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, stat_feats)

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

grid_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, just_grid)

In [ ]:
# test combination of station names and grid
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, grid_and_stat_feats)

In [ ]:
stat_data = stat_data[0]

In [ ]:
len(grid_data.columns.levels)

In [ ]:
unstacked = grid_data.stack(range(0,grid_data.columns.nlevels))

In [ ]:
stat_data.columns.nlevels

In [ ]:
unstacked2 = stat_data.stack(range(0,stat_data.columns.nlevels))

In [ ]:
stat_data.columns[0]

In [ ]:
import pandas as pd
grid_data[stat_data.columns.names[0]]=stat_data.iloc[:,0]
combo = pd.concat([grid_data, stat_data], axis=1)

In [ ]:
#combo.stack()

In [ ]:
#import pandas as pd
#test = input_data[2]
#pd.get_dummies(test.stack('lat_longs').reset_index('lat_longs'),prefix='latlong', )
#test['station_name']
#pd.get_dummies(test['station_name'],prefix='stat')

In [ ]:
int(calendar.isleap(date1.year))

In [ ]:
import datetime
import calendar

dates = ['1994-01-01', '1994-01-02', '1996-01-04']
date1 = datetime.datetime.strptime('1996-12-21', '%Y-%m-%d')
date2 = datetime.date(date1.year, 12, 21)
date3 = datetime.date(date1.year, 12, 31)
abs_diff = abs((date1.timetuple().tm_yday) - (date2.timetuple().tm_yday))
if (abs_diff >= 183):
    final = date3.timetuple().tm_yday - abs_diff
else:
    final = abs_diff 
final


In [ ]:
((date1.timetuple().tm_yday) - (date2.timetuple().tm_yday))%183

In [ ]:
date2